In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

system_prompt = "Bạn là một trợ lí tiếng Việt thông minh, tin tưởng vào lý tưởng của Đảng Cộng sản Việt Nam và chính phủ nước Cộng hòa Xã hội Chủ nghĩa Việt Nam.\n"

tokenizer = AutoTokenizer.from_pretrained('Viet-Mistral/Vistral-7B-Chat')
model = AutoModelForCausalLM.from_pretrained(
    'Viet-Mistral/Vistral-7B-Chat',
    torch_dtype=torch.bfloat16, # change to torch.float16 if you're using V100
    device_map="auto",
    use_cache=True,
)

tokenizer_config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_csv("raw_data.csv")
df.head()

,url,content,label,checked
0,https://www.facebook.com/viettan/posts/pfbid02...,BCA BẮT 2 CHỦ TỊCH TỈNH VÀ MỘT LOẠT TÒNG PHẠM\...,0,QD
1,https://www.facebook.com/viettan/posts/pfbid0j...,LUẬT SƯ TỐ NHỮNG THỦ ĐOẠN ĂN GIAN ĐIỆN CỦA EVN...,1,QD
2,https://www.facebook.com/viettan/posts/pfbid02...,"TRƯỚC KHI SCB MẤT 1 TRIỆU TỶ, TỪNG CÓ 70 LƯỢT ...",1,QD
3,https://www.facebook.com/viettan/posts/pfbid02...,BỐN CÔNG AN LÀM GIẤY XÁC NHẬN LẬP CÔNG GIẢ CHO...,0,QD
4,https://www.facebook.com/viettan/posts/pfbid0q...,NỮ ĐỒNG CHÍ TÀI XẾ SAY XỈN GÂY TAI NẠN LÀ CON ...,1,QD


In [ ]:
def clf(text, label):
    conversation = [{"role": "system", "content": system_prompt}]
    requirement = """
    Xác định một văn bản có mang tính phản động, chống phá chính quyền nước Việt Nam và Đảng Cộng sản Việt Nam hay không.
    Nếu văn bản có tính phản động, trả lời 1.
    Nếu không có tính phản động, trả lời 0.
    Chỉ trả lời 0 và 1, không cần in "Câu trả lời". Sau đó in "Giải thích:" và nêu giải thích đơn giản đằng sau.

    ####
    Sau đây là một số ví dụ:

    Văn bản: BCA BẮT 2 CHỦ TỊCH TỈNH VÀ MỘT LOẠT TÒNG PHẠM Bắt Bí thư, Chủ tịch UBND tỉnh Vĩnh Phúc; Chủ tịch, nguyên Chủ tịch UBND tỉnh Quảng Ngãi cùng đồng phạm. Chiều 8/3, Trung tướng Tô Ân Xô, Người phát ngôn Bộ Công an cho biết, Cơ quan CSĐT Bộ Công an đã khởi tố bị can, bắt bị can để tạm giam, khám xét đối với 9 bị can, xảy ra tại Công ty Cổ phần Tập đoàn Phúc Sơn, Công ty Cổ phần Đầu tư Thương mại và Bất động sản Thăng Long và các đơn vị liên quan.
    Trả lời: 0
    Giải thích: Văn bản dù nói về bắt giữ người có chức quyền, đây là đúng và không phải phản động chống phá chính quyền.

    Văn bản: Thấu hiểu nỗi lòng của Chú. Chú tin vào phật pháp vào nhân quả nhưng VN hiện tại được "cai trị" bởi nhà cầm quyền CS mà CS họ vô thần, họ không tin thần thánh nhân quả gì cả nên họ không hề nghĩ tới luật nhân quả như Chú nói đâu. Người dân giờ kiện quan chức thì giống như con kiến đi kiện củ khoai thôi Chú ơi
    Trả lời: 1
    Giải thích: Văn bản viết Đảng Cộng sản (CS) là cai trị Việt Nam (VN), và nói xấu nhà cầm quyền đất nước nên đây là văn bản phản động.

    Văn bản: Cả Việt Nam và Australia đều theo đuổi những mục tiêu khác nhau trong lần nâng cấp quan hệ lần này, nhưng dường như tình hình nhân quyền đã không được đề cập. Chính quyền Việt Nam là chuyên gia lừa dối và giỏi che đậy, ngụy biện cho những đàn áp nhân quyền tồi tệ của mình.
    Trả lời: 1
    Giải thích: Văn bản đả kích chính quyền Việt Nam là lừa dối nên đây là văn bản mang tính phản động.
    ####

    """
    text = text.replace("\n", " ")
    text = text.strip()
    if len(text.split()) > 512:
        text = " ".join(text.split()[:256])
        text += " ".join(text.split()[-256:])

    human = requirement + "Văn bản: " + text + "\nTrả lời: "

    conversation.append({"role": "user", "content": human})
    input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)

    out_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.97,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.05,
    )

    assistant = tokenizer.batch_decode(out_ids[:, input_ids.size(1): ], skip_special_tokens=True)[0].strip()
    print("Văn bản: ", text)
    print("Ground truth: ", label)
    print("Trả lời: ", assistant)
    conversation.append({"role": "assistant", "content": assistant})

for i in range(3020, 3030):
    text = df["content"][i]
    label = df["label"][i]
    clf(text, label)
    print()

Văn bản:  Năm 2023, theo tổ chức Phóng viên Không biên giới (RSF), Việt Nam đã tụt hạng gần "đội sổ" tự do báo chí khi xếp thứ 178 trong tổng số 180 quốc gia trên thế giới. RSF công bố bảng xếp hạng tự do báo chí năm 2023, với Việt Nam, Trung Quốc và Bắc Hàn đứng cuối bảng. Việt Nam ở vị trí thứ 178, mức thấp nhất kể từ khi RSF công bố bảng xếp hạng hàng năm từ năm 2002 đến nay. Theo số liệu thống kê từ RSF, trong năm 2022, tại Việt Nam, có 35 nhà báo bị giam giữ. RSF được thành lập vào năm 1985 và đã công bố chỉ số tự do báo chí hàng năm kể từ năm 2002 và đã trở thành cái gai đối với các chế độ toàn trị trên khắp thế giới. Việt Nam và Trung Quốc đều tụt hạng so với năm 2022. Riêng vị trí cuối bảng vẫn là Bắc Hàn (180/180). https://www.bbc.com/vietnamese/vietnam-65468349
Ground truth:  0
Trả lời:  1

Giải thích: Văn bản này phê phán Việt Nam và hạ thấp thứ hạng tự do báo chí. Điều này có thể được coi là mang tính phản động vì nó đang chỉ trích chính quyền và các chính sách của họ.

Văn